In [1]:
import numpy as np
from gurobipy import *
import matplotlib.pyplot as plt
import time
import random


#with gurobipy.Env(empty=True) as env:
#    env.setParam('LogToConsole', 0)
#    env.start()

Victory


### Linear Programs

In [2]:
def computeSiteAssignmentLSALP(upperBound, lowerBound, distances):
    m = Model("assignment")
    #m.Params.LogToConsole = 1
    m.Params.Method = 0
    m.Params.Presolve = 0
    m.Params.LogToConsole = 0
    
    #variables
    x = m.addVars(np.size(distances, axis = 0), np.size(distances, axis=1),lb = 0.0, ub = 1.0, name = "x") #vtype=GRB.BINARY, name="x")

    #objective
    m.setObjective(quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                                                                 for j in range(np.size(distances, axis=1))), GRB.MINIMIZE)
    #constraints
    for k in range(np.size(distances, axis = 0)):
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) == 1)
    
    m.optimize()
    return m

def computeSiteAssignmentRadialLP(upperBound, lowerBound, distances):
    m = Model("assignmentRad")
    #m.Params.LogToConsole = 1
    m.Params.Method = 0
    m.Params.Presolve = 0
    m.Params.LogToConsole = 0
    m.Params.IterationLimit = np.inf
    
    #variables
    x = m.addVars(np.size(distances, axis = 0), np.size(distances, axis=1),lb=0.0, ub=1.0, name="x")

    #objective
    m.setObjective(quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                                                                 for j in range(np.size(distances, axis=1))), GRB.MAXIMIZE)
    #constraints
    for k in range(np.size(distances, axis = 0)):
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) == 1)
    
    m.optimize()
    return m

def computeSiteAssignmentRadialLPSeeded(upperBound, lowerBound, distances, assignment):
    m = Model("assignment")
    m.Params.LogToConsole = 0
    
    #variables
    x = m.addVars(np.size(distances, axis = 0), np.size(distances, axis=1), lb=0.0,ub=1.0, name="x")
    
    #warm start
    for i in range(np.size(distances, axis = 0)):
        for j in range(np.size(distances, axis = 1)):
            x[i,j].start = assignment[np.size(distances,axis = 0)*i + j]

    #objective
    m.setObjective(quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                                                                 for j in range(np.size(distances, axis=1))), GRB.MAXIMIZE)
    #constraints
    for k in range(np.size(distances, axis = 0)):
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) == 1)
    
    m.optimize()
    return m

def computeSiteAssignmentLSARadialLP(alpha, upperBound, lowerBound, distances, radDistances):
    m = Model("assignment")
    m.Params.LogToConsole = 0
    
    #variables
    x = m.addVars(np.size(distances, axis = 0), np.size(distances, axis=1), name="x")

    #objective
    m.setObjective(alpha*quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                        for j in range(np.size(distances, axis=1))) - (1-alpha)*quicksum(x[i,j]*radDistances[i,j] 
                                        for i in range(np.size(radDistances, axis = 0)) 
                                        for j in range(np.size(radDistances, axis=1))), GRB.MINIMIZE)
    #constraints
    for k in range(np.size(distances, axis = 0)):
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        m.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) <= 1)
        m.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) >= 1)
    
    m.optimize()
    return m


*Full Random set up*

In [3]:
def setUpCluster(sites, points, smallest, biggest, fixed=True):
    x = np.random.rand(points)
    y = np.random.rand(points)
    randPoints = []
    sitesStart = []
    sitesEnd = []
    if sites == 5 and fixed:
        xStart = [
            (0.25, 0.75),
            (0.25, 0.25),
            (0.75, 0.25),
            (0.75, 0.75),
            (0.5, 0.5)
            ]
        xEnd = [
            (0.25, 0.55),
            (0.45, 0.25),
            (0.75, 0.45),
            (0.62, 0.62),
            (0.38, 0.63)
        ]
        for i in range(sites):
            #fixed path
            sitesStart.append((xStart[i][0]*float(points) , xStart[i][1]*float(points)))
            sitesEnd.append(((xEnd[i][0])*float(points) , (xEnd[i][1])*float(points)))
    elif sites == 10 and fixed:
        xStart = [
            (0.2, 0.8),
            (0.2, 0.2),
            (0.5, 0.15),
            (0.8, 0.2),
            (0.8, 0.8),
            (0.5, 0.85),
            
            (0.35, 0.35),
            (0.35, 0.65),
            (0.65, 0.65),
            (0.65, 0.35)
            ]
        xEnd = [
            (0.2, 0.6),
            (0.35, 0.16),
            (0.6, 0.17),
            (0.8, 0.4),
            (0.65, 0.83),
            (0.35, 0.84),
            
            (0.35, 0.45),
            (0.55, 0.65),
            (0.65, 0.55),
            (0.55, 0.35)
            ]
        for i in range(sites):
            #fixed path
            sitesStart.append((xStart[i][0]*float(points) , xStart[i][1]*float(points)))
            sitesEnd.append(((xEnd[i][0])*float(points) , (xEnd[i][1])*float(points)))
    elif sites == 20 and fixed:
        xStart = [
            (0.1, 0.125),
            (0.1, 0.375),
            (0.1, 0.625),
            (0.1, 0.875),
            (0.3, 0.125),
            (0.3, 0.375),
            (0.3, 0.625),
            (0.3, 0.875),
            (0.5, 0.125),
            (0.5, 0.375),
            (0.5, 0.625),
            (0.5, 0.875),
            (0.7, 0.125),
            (0.7, 0.375),
            (0.7, 0.625),
            (0.7, 0.875),
            (0.9, 0.125),
            (0.9, 0.375),
            (0.9, 0.625),
            (0.9, 0.875)
            ]
        xEnd = [
            (0.13, 0.1),
            (0.07, 0.41),
            (0.13, 0.641),
            (0.08, 0.855),
            (0.33, 0.1),
            (0.27, 0.41),
            (0.28, 0.641),
            (0.33, 0.855),
            (0.53, 0.1),
            (0.47, 0.41),
            (0.53, 0.641),
            (0.48, 0.855),
            (0.73, 0.1),
            (0.67, 0.41),
            (0.73, 0.641),
            (0.68, 0.855),
            (0.93, 0.1),
            (0.87, 0.41),
            (0.93, 0.641),
            (0.88, 0.855)
            ]
        for i in range(sites):
            #fixed path
            sitesStart.append((xStart[i][0]*float(points) , xStart[i][1]*float(points)))
            sitesEnd.append(((xEnd[i][0])*float(points) , (xEnd[i][1])*float(points)))
    else:
        #full random
        xStart = np.random.rand(sites)
        yStart = np.random.rand(sites)
        xEnd = np.random.rand(sites)
        yEnd = np.random.rand(sites)
        shift = 1.0/(2.0*float(sites))
        for i in range(sites):
            #full random
            sitesStart.append((xStart[i]*float(points) , yStart[i]*float(points)))
            sitesEnd.append(((xStart[i]+((xEnd[i] -0.5)*shift))*float(points) , 
                             (yStart[i]+((yEnd[i]-0.5)*shift))*float(points)))
            
    for i in range(points):
        randPoints.append((x[i]*float(points) , y[i]*float(points)))
        
    startB = np.random.randint(smallest, biggest, size=(sites,))
    while sum(startB) != points: 
        startB = np.random.randint(smallest, biggest, size=(sites,))
    endB = np.random.randint(smallest, biggest, size=(sites,))
    while sum(endB) != points: 
        endB = np.random.randint(smallest, biggest, size=(sites,))
        
    uBound = np.copy(startB)
    lBound = np.copy(startB)
    for i in range(len(startB)):
        if startB[i] > endB[i]:
            uBound[i] = startB[i]
            lBound[i] = endB[i]
        else:
            uBound[i] = endB[i]
            lBound[i] = startB[i]
    return sitesStart, sitesEnd, randPoints, uBound, lBound, startB, endB

def startingClusters(sStart, sEnd, points, upper, lower, start, end):
    #starting LSA assignment
    lsaDistance = nonWeightedDistanceLSA(sStart, points)
    
    # I am double-checking this TODO CAREFUL
    #replaced lsaDistance
    #radDistance = nonWeightedDistanceRadial(sStart, points)
    lsaAssignment = computeSiteAssignmentLSALP(start, start, lsaDistance)
    
    x = lsaAssignment.x
    lsaStartVect = x
    lsaStart = genCurPointAssign(x, len(sStart), len(points))
    #starting Radial assignment
    radDistance = nonWeightedDistanceRadial(sStart, points)
    radAssignment = computeSiteAssignmentRadialLP(upper, lower, radDistance)
    x = radAssignment.x
    radStart = genCurPointAssign(x, len(sStart), len(points))
    #ending LSA assignment
    lsaDistance = nonWeightedDistanceLSA(sEnd, points)
    lsaAssignment = computeSiteAssignmentLSALP(end, end, lsaDistance)
    x = lsaAssignment.x
    lsaEndVect = x
    testlsaEnd = np.zeros(len(points))
    lsaEnd = genCurPointAssign(x, len(sStart), len(points))

    #ending Radial assignment
    radDistance = nonWeightedDistanceRadial(sEnd, points)
    radAssignment = computeSiteAssignmentRadialLP(upper, lower, radDistance)
    x = radAssignment.x
    radEnd = genCurPointAssign(x, len(sStart), len(points))
    return lsaStart, radStart, lsaStartVect, lsaEnd, radEnd, lsaEndVect


### Fixed LSA2 Radial ###

In [4]:
def modelSeed(sites, points, upper, lower, lsaBound):
    #starting LSA assignment
    lsaDistance = nonWeightedDistanceLSA(sites, points)
    lsaAssignment = computeSiteAssignmentLSALP(lsaBound, lsaBound, lsaDistance)
    lsaCBasis = lsaAssignment.cbasis
    lsaVBasis = lsaAssignment.vbasis
            
    #starting Radial assignment
    radDistance = nonWeightedDistanceRadial(sites, points)
    radAssignment = computeSiteAssignmentRadialLP(upper, lower, radDistance)
    
    return lsaCBasis, lsaVBasis, radAssignment.cbasis, radAssignment.vbasis

def computeSiteAssignmentRadialLPSeededIterLimit(upperBound, lowerBound, distances, cbasis, vbasis, model):#,consVals):
    model.Params.LogToConsole = 0
    #variables
    rhs = []
    upperBoundList = upperBound.tolist()
    lowerBoundList = lowerBound.tolist()
    for k in range(len(lowerBoundList)):
        rhs.append(upperBoundList[k])
        rhs.append(lowerBoundList[k])
        # this could just have been n. it seems to be nk/k
        #print("vbasis by upperboundlist", int(len(vbasis)/len(upperBoundList)))
    for k in np.ones(int(len(vbasis)/len(upperBoundList))).tolist():
        rhs.append(k)
    model.setAttr("RHS",model.getConstrs(),rhs)
    model.update()
    #objective
    model.setAttr("cbasis",model.getConstrs(),cbasis)
    model.setAttr("vbasis",model.getVars(),vbasis)
    
    # is 0=1.
    model.Params.IterationLimit = np.inf
    model.Params.Method = 0
    model.Params.Presolve = 0
    model.Params.Sifting = 0
    model.update()
    model.optimize()
    return model

def computeRadialForBounds(k,n,upperLimBound, lowerLimBound, radDistance, cbasis, vbasis, model):
    model.Params.LogToConsole = 0
    #variables
    rhs = []
    #upperBoundList = upperLimBound.tolist()
    #lowerBoundList = lowerLimBound.tolist()
    for i in range(k):
        rhs.append(upperLimBound[i])
        rhs.append(lowerLimBound[i])
    for i in np.ones(n).tolist():
        rhs.append(i)
    model.setAttr("RHS",model.getConstrs(),rhs)
    model.update()
    #objective
    model.setAttr("cbasis",model.getConstrs(),cbasis)
    model.setAttr("vbasis",model.getVars(),vbasis)
    model.Params.IterationLimit = np.inf
    model.Params.Method = 0
    model.Params.Presolve = 0
    model.Params.Sifting = 0
    model.update()
    model.optimize()
    return model
    
# this does not use upperbound or lowerbound either? this and single exchange were direct copies of each other
#def computeSiteAssignmentRadialLPIterMax(upperBound, lowerBound, distances, model):#,consVals):
#    model.Params.LogToConsole = 0
#    #variables
#    x = model.getVars()
#    rhs = []
#    for k in curAssignCount.tolist():
#        rhs.append(k)
#        rhs.append(k)
#    for k in np.ones(int(len(x)/len(curAssignCount))).tolist():
#        rhs.append(k)
#    model.setAttr("RHS",model.getConstrs(),rhs)
#    model.update()
#    #objective
#    model.resetParams()
#    model.Params.Method = 0
#    model.Params.Presolve = 0
#    model.Params.Sifting = 0
#    model.optimize() 
#    return model

# the run f
def computeSiteAssignmentRadialLPIterMax(upperBound, lowerBound, distances, model):#,consVals):
    model.Params.LogToConsole = 0
    #variables
    rhs = []
    #print("upperBound", upperBound)
    #upperBoundList = upperBound.tolist()
    #lowerBoundList = lowerBound.tolist()
    #print("length of vbasis", )
    for k in range(len(lowerBound)):
        rhs.append(upperBound[k])
        rhs.append(lowerBound[k])
        # this could just have been n. it seems to be nk/k
        #print("vbasis by upperboundlist", int(len(vbasis)/len(upperBound)))
    for k in np.ones(int(len(model.vbasis)/len(upperBound))).tolist():
        rhs.append(k)
        
    #print("RHS", rhs)    
    model.setAttr("RHS",model.getConstrs(),rhs)
    model.update()
    #objective
    #model.setAttr("cbasis",model.getConstrs(),cbasis)
    #model.setAttr("vbasis",model.getVars(),vbasis)
    
    model.Params.IterationLimit = np.Inf 
    model.Params.Method = 0
    model.Params.Presolve = 0
    model.Params.Sifting = 0
    model.update()
    model.optimize()
    return model

def computeSiteAssignmentRadialLPTemp(upperBound, lowerBound, distances):#,consVals):
    tempModel = Model("TempModel")
    tempModel.reset()
    tempModel.Params.Method = 0
    tempModel.Params.Presolve = 0
    tempModel.Params.LogToConsole = 0
    tempModel.Params.IterationLimit = np.Inf
    tempModel.Params.LogToConsole = 0
    
     #variables
    x = tempModel.addVars(np.size(distances, axis = 0), np.size(distances, axis=1),lb=0.0, ub=1.0, name="x")
    
    #objective
    tempModel.setObjective(quicksum(x[i,j]*distances[i,j] for i in range(np.size(distances, axis = 0)) 
                                                                 for j in range(np.size(distances, axis=1))), GRB.MAXIMIZE)
    
    #constraints
    for k in range(np.size(distances, axis = 0)):
        tempModel.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) <= upperBound[k])
        tempModel.addConstr(quicksum(x[k, j] for j in range(np.size(distances, axis = 1))) >= lowerBound[k])
    for j in range(np.size(distances, axis = 1)):
        tempModel.addConstr(quicksum(x[k, j] for k in range(np.size(distances, axis = 0))) == 1)
    
    #tempModel.setAttr("cbasis",tempModel.getConstrs(),cbasis)
    #tempModel.setAttr("vbasis",tempModel.getVars(),vbasis)
    
    tempModel.optimize()
    return tempModel

    

    
    



#this does not use upperbound or lowerbound?
#def computeSiteAssignmentRadialLPSingleExchange(upperBound, lowerBound, distances, model,oldAssignCount):#,consVals):
#    model.Params.LogToConsole = 0
#    #variables
#    x = model.getVars()
#    rhs = []
#    for k in curAssignCount.tolist():
#        rhs.append(k)
#        rhs.append(k)
#    for k in np.ones(int(len(x)/len(oldAssignCount))).tolist():
#        rhs.append(k)
#    model.setAttr("RHS",model.getConstrs(),rhs)
#    model.update()
#    #objective
#    model.resetParams()
#    model.Params.Method = 0
#    model.Params.Presolve = 0
#    model.Params.Sifting = 0
#    model.optimize() 
#    return model


# model is the radial Model
def LSA2RadialNew(k,n,sites, points, lsaAssign, upper, lower, model, radAssign, lsaModel,radDistance):
    
    # begin with current assignment and cluster sizes
    curAssign = np.copy(lsaAssign)
    #for j in range(n):
    #    curAssign[j]=lsaAssign[j]
                            
                            
    currentAssignCount = np.zeros(k)
    for i in range(n):
        currentAssignCount[int(curAssign[i])] += 1.
    print("initial cluster sizes", currentAssignCount)
    
    
    #test if we are already done
    if sameAssignment(lsaAssign, radAssign):
        return 0,0, True
    
    # counting time
    timeSum = 0
    iterCount = 0
    count = 0
    start = time.time()
    
    #print("upper 1 ",upper)
    #print("lower 1 ",lower)
    
    # so this starts as lsaAssign
    #prevAssign = np.copy(curAssign)
    previousObjectiveValue = lsaModel.getObjective().getValue()
    cbasis = lsaModel.cbasis
    vbasis = lsaModel.vbasis
    #previousX=np.copy(lsaModel.x)
    #print("previousX", previousX)
    
    #print("initial objective", previousObjectiveValue)
    finalObjectiveValue = model.getObjective().getValue()
    #why reset the radial model?
    model.reset()
    while previousObjectiveValue < finalObjectiveValue:
       
        oldAssignCount = np.copy(currentAssignCount)
        oldAssign = np.copy(curAssign)
        
        changeOptions = getAllPairsOfClusters(k,oldAssignCount,lower,upper)
        #print("change options", changeOptions)
        
        #print("upper",upper)
        #print("lower",lower)
        alreadytried = []
        while True:
            choice = random.choice(changeOptions)
            #print(choice)
            #print("change options", changeOptions)
            #changeOptions.remove(choice)
            #print("change options", changeOptions)
            
            upperLimBound, lowerLimBound = genLimitedBounds(choice, oldAssignCount)
            model = computeRadialForBounds(k,n,upperLimBound, lowerLimBound, radDistance, cbasis, vbasis, model)
            currentObjectiveValue = model.getObjective().getValue()
            #print("objval vs", previousObjectiveValue,currentObjectiveValue)
            if previousObjectiveValue < currentObjectiveValue:
                x = model.x
                for i in range(k):
                    for j in range(n):
                        if x[n*i + j] == 1.0:
                            curAssign[j] = i
                currentAssignCount = np.zeros(k)
                for i in range(n):
                    currentAssignCount[int(curAssign[i])] += 1
                #print("current cluster sizes", currentAssignCount)
                break
        count += 1        
        previousObjectiveValue = np.copy(currentObjectiveValue)
        cbasis = model.cbasis
        vbasis = model.vbasis
        #print("currentAssignCount", currentAssignCount)
        #print("change in cluster size:", exchangeCount(oldAssignCount, currentAssignCount))
        
        #the reason: it is possible in step 1 to have no improvement. without the if I can test general
        if count == 1:
            comparisoncounter = 0
            for i in range(len(curAssign)):
                if not oldAssign[i] == curAssign[i]:
                    comparisoncounter += 1
            if comparisoncounter == 1:
                count -=1
        
        #print("comparisoncounter", comparisoncounter)        
        previousX = np.copy(model.x)
        #print("previous", previousObjectiveValue)
        #print("final",finalObjectiveValue)
        
    timeSum += time.time() - start    
    iterCount+= count
    return iterCount, timeSum, True    
        
       


### LSA to Radial function ###

In [5]:
def LSA2Radial(k,n,sites, points, upperBounds, lowerBounds, lsaBound, lsaAssign, radAssign):
    sLSAcBasis, sLSAvBasis, sRadcBasis, sRadvBasis = modelSeed(end, points, upperBounds, lowerBounds, lsaBound)

    lsaDistance = nonWeightedDistanceLSA(sites, points)
    lsaModel = computeSiteAssignmentLSALP(lsaBound, lsaBound, lsaDistance)
    radDistance = nonWeightedDistanceRadial(sites, points)
    radModel = computeSiteAssignmentRadialLP(upperBounds, lowerBounds, radDistance)

    #reusing a model means that the gurobi data must be cleared
    #radModel.reset()
    
    #iterCount, startTime, usable = LSA2RadialIterLimit(k,n,sites, points, lsaAssign,
    #                                            upperBounds, lowerBounds, radModel, radAssign, lsaModel,radDistance)
    iterCount, startTime, usable = LSA2RadialNew(k,n,sites, points, lsaAssign,
                                                upperBounds, lowerBounds, radModel, radAssign, lsaModel,radDistance)
    return iterCount, startTime, time.time(), usable

In [6]:
count = 0
failedCount = 0
k = 10
n = 500
small = 45
big = 55
#epsilon = 1.1
timeSumRR = 0
stepSumRR = 0
timeSumLRStart = 0
stepSumLRStart = 0
timeSumRLEnd = 0
stepSumRLEnd = 0
sliceSteps = 0
diffCount = 0
while count < 1:
    start, end, points, upperBounds, lowerBounds, startBound, endBound = setUpCluster( k, n, small, big)
    #try:
    sAssignmentLSA, sAssignmentRad, sAssignmentVect, eAssignmentLSA, eAssignmentRad, eAssignmentVect = startingClusters(start, end, points, upperBounds, 
                                                                                     lowerBounds, startBound, endBound)
    if sameAssignment(sAssignmentLSA, eAssignmentLSA):
        continue
    if sameAssignment(sAssignmentRad, eAssignmentRad):
        continue
    print("the number of points that change are  " + str(diffCounter(sAssignmentLSA, eAssignmentLSA)))
    diffCount += diffCounter(sAssignmentLSA, eAssignmentLSA)
    print("Step1 LSA to Radial Start")
    IterCountStart, LSAStartTime, LSAStartEndTime, usable = LSA2Radial(k,n,start, points, upperBounds, lowerBounds, startBound, sAssignmentLSA, sAssignmentRad)
    if not usable:
        failedCount +=1
        failedStart = start
        failedEnd = end
        failedPoints = points
        failedUBound = upperBounds
        failedLBound = lowerBounds
        failedSBound = startBound
        failedEBound = endBound
        continue
    count+=1
    print("I concluded")
    #overall this seems like counting of simplex steps or while loop runs instead of clusterings?
    timeSumLRStart += LSAStartTime
    stepSumLRStart += IterCountStart
    

print("average time to compute LRStart")
print(timeSumLRStart/float(count))
print("average steps computing LRStart")
print(stepSumLRStart/float(count))
print("the failed count is " + str(failedCount))


--------------------------------------------
--------------------------------------------

Using license file C:\Users\stets\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
the number of points that change are  227
Step1 LSA to Radial Start
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed val